In [121]:
import pandas as pd
import numpy as np
from pandas.core.frame import DataFrame
from tqdm import tqdm
import math
import pymssql
import csv
import matplotlib.pyplot as plt
import time
import seaborn as sns

In [110]:
f1 = open('/Eurekalert_research_level.csv',encoding='utf-8-sig')
data_rl = pd.read_csv(f2,sep=';')
f1.close()

In [ ]:
#Figure 1
plt.figure(figsize=(10, 6))
sns.set(style="whitegrid") 
#colors

color2 = [52/255.0, 108/255.0,172/255.0]

color4 = [247/255.0, 228/255.0,116/255.0]

rgb_list = [color2,color4]

#data
grouped_data = data_rl.groupby(['post_time', 'RL_number'])['euid'].count( ).reset_index()

# distribution over time
i = 0
for category, data in grouped_data.groupby('RL_number'):
    x = data['post_time']
    y = data['euid']
    plt.plot(x[0:8], y[0:8], label=category,linewidth =4 ,color = rgb_list[i] ) #,color = rgb_list[i]
    i = i+1

    
handles, labels = plt.gca().get_legend_handles_labels()
labels = ['Basic research','Applied research']
#handles = [handles[1],handles[0]]


plt.legend(handles,labels)


plt.xlabel('Year of publication')
plt.ylabel('Number of press releases')

plt.show() 

In [ ]:
#figure 2
RL1 = data_rl[data_rl['LR_main_field'] == 'Mathematics and computer science']
grouped_data1 = RL1.groupby(['post_time', 'RL_number'])['euid'].count( ).reset_index()

RL2 = data_rl[data_rl['LR_main_field'] == 'Biomedical and health sciences']
grouped_data2 = RL2.groupby(['post_time', 'RL_number'])['euid'].count( ).reset_index()

RL3 = data_rl[data_rl['LR_main_field'] == 'Physical sciences and engineering']
grouped_data3 = RL3.groupby(['post_time', 'RL_number'])['euid'].count( ).reset_index()

RL4 = data_rl[data_rl['LR_main_field'] == 'Social sciences and humanities']
grouped_data4 = RL4.groupby(['post_time', 'RL_number'])['euid'].count( ).reset_index()

RL5 = data_rl[data_rl['LR_main_field'] == 'Life and earth sciences']
grouped_data5 = RL5.groupby(['post_time', 'RL_number'])['euid'].count( ).reset_index()

fig = plt.figure(figsize=(15, 8))  # create a figure

labels = ['Basic research','Applied research']
#labels = ['Applied technology','Engineering-technological mix','Applied research','Basic research']
# 1， 3
ax1 = fig.add_subplot(2, 3, 3)  # (行数, 列数, 位置索引)
i = 0
for category, data in grouped_data1.groupby('RL_number'):
    x = data['post_time']
    y = data['euid']
    ax1.plot(x[0:8], y[0:8], label=category,linewidth =4,color = rgb_list[i] )#,color = rgb_list[i] )
    i = i+1
ax1.legend(labels) 
ax1.set_xlabel('Year of publication')
ax1.set_ylabel('Number of press releases')
ax1.set_title('Mathematics and computer science')

# 1，1 
ax2 = fig.add_subplot(2, 3, 1)
i = 0
for category, data in grouped_data2.groupby('RL_number'):
    x = data['post_time']
    y = data['euid']
    ax2.plot(x[0:8], y[0:8], label=category,linewidth =4,color = rgb_list[i] )#,color = rgb_list[i] )
    i = i+1
ax2.legend(labels) 
ax2.set_xlabel('Year of publication')
ax2.set_ylabel('Number of press releases')
ax2.set_title('Biomedical and health sciences')

# 2，1
ax3 = fig.add_subplot(2, 3, 4)
i = 0
for category, data in grouped_data3.groupby('RL_number'):
    x = data['post_time']
    y = data['euid']
    ax3.plot(x[0:8], y[0:8], label=category,linewidth =4,color = rgb_list[i] )# ,color = rgb_list[i] )
    i = i+1
ax3.legend(labels)
ax3.set_xlabel('Year of publication')
ax3.set_ylabel('Number of press releases')
ax3.set_title('Physical sciences and engineering')

# 1，2
ax4 = fig.add_subplot(2, 3, 2)
i = 0
for category, data in grouped_data4.groupby('RL_number'):
    x = data['post_time']
    y = data['euid']
    ax4.plot(x[0:8], y[0:8], label=category,linewidth =4,color = rgb_list[i])#,color = rgb_list[i] )
    i = i+1
ax4.legend(labels)
ax4.set_xlabel('Year of publication')
ax4.set_ylabel('Number of press releases')
ax4.set_title('Social sciences and humanities')

# 2，2
ax5 = fig.add_subplot(2, 3, 5)
i = 0
for category, data in grouped_data5.groupby('RL_number'):
    x = data['post_time']
    y = data['euid']
    ax5.plot(x[0:8], y[0:8], label=category,linewidth =4,color = rgb_list[i])# ,color = rgb_list[i] )
    i = i+1
ax5.legend(labels) 
ax5.set_xlabel('Year of publication')
ax5.set_ylabel('Number of press releases')
ax5.set_title('Life and earth sciences')

# 显示图形
plt.tight_layout()
plt.show()

In [ ]:
----------------------------------calculate the title TFIDF similarity---------------------

In [58]:
sql = "select * from eu_openalex_LR"# a inner join dimention_abstract_title b on a.euid = b.euid where abstract like '%background:%' or abstract like '%Methods:%'  or abstract like '%purpose:%'"#"select * from [userdb_zhangz10].[dbo].[dimention_abstract_title]"
data = sql_df(sql)

In [ ]:
##tfidf model
df= pd.DataFrame()
df['Merged_eu'] = data['eurekalert_title']   +' ' + data['content']
df['Merged_paper']= data['paper_title']   +' ' + data['abstract']

def remove_number(text_list):
    test_remove = []
    for text in text_list:
        test_remove.append(re.sub(r'\d', '', text).replace('_', ''))
    return test_remove

train = df['Merged_eu'].tolist() + df['Merged_paper'].tolist()
train = remove_number(train)
mytime = time.time()
stop_words = set(stopwords.words('english'))
tfidf_model = TfidfVectorizer(max_df = 0.995,min_df=0.005,stop_words="english").fit(train)
print(time.time()-mytime)
print(len(tfidf_model.get_feature_names_out()))

In [ ]:
#caculat the title similarity by cosine
title_similarity = [['euid','cosine_similarity']]
file1 = open('/title_similarity_cosine.csv', 'a+', newline='', encoding='utf-8')
writer1 = csv.writer(file1, delimiter=';')
writer1.writerows( title_similarity)
file1.close() 

for i in tqdm(range(0,len(data['euid']))):
    euid = data['euid'][i]
    eu_title = data['eurekalert_title'][i]
    paper_title = data['paper_title'][i]


    tfidf_matrix = tfidf_model.transform([eu_title, paper_title])
    similarity = cosine_similarity(tfidf_matrix[1], tfidf_matrix[0])

    cosine_sim = round(similarity[0][0],2)

    
    title_similarity = [[euid,cosine_sim]]
    file1 = open('/title_similarity_cosine.csv', 'a+', newline='', encoding='utf-8')
    writer1 = csv.writer(file1, delimiter=';')
    writer1.writerows( title_similarity)
    file1.close()

In [114]:
f1 = open('/title_similarity_cosine.csv',encoding='latin1')
title_sim = pd.read_csv(f1,sep=';')
f1.close()
rl_si = pd.merge(data_rl,title_sim, on='euid', how='inner' )

In [ ]:
#Figure 5
# 设置 Seaborn 风格
sns.set(style="ticks") #style="whitegrid"
color1 = [14/255.0, 62/255.0,135/255.0]
color2 = [52/255.0, 108/255.0,172/255.0]
color3 = [222/255.0, 234/255.0,234/255.0] 
color4 = [247/255.0, 228/255.0,116/255.0]
color5 = [216/255.0, 178/255.0,58/255.0]
rgb_list = [color1,color2,color3,color4,color5]
# 使用 Seaborn 画箱线图
plt.figure(figsize=(12, 6))
ax = sns.boxplot(hue = 'RL_number',x = 'post_time', y='cosine_similarity', data=rl_si,palette= [rgb_list[1],rgb_list[3]], linewidth=1,showmeans=True,medianprops= dict(color= rgb_list[0]),meanprops = {'marker':'d','markerfacecolor':rgb_list[4],'markeredgecolor':rgb_list[4]})#'markerfacecolor':'b'
#ax.set_xticklabels(['Basic research','Applied research'])
ax.set_xticklabels(ax.get_xticklabels())
    
handles, labels = plt.gca().get_legend_handles_labels()
labels = ['Basic research','Applied research']
handles = [handles[0],handles[1]]


#plt.legend(loc = 'upper right')
plt.legend(handles,labels,loc = 'upper right')

plt.xlabel('Year of publication')
plt.ylabel('Similarity of titles between papers and press releases')
plt.show()

In [ ]:
#Figure 6
# 设置 Seaborn 风格
sns.set(style="ticks") #style="whitegrid"
# 使用 Seaborn 画箱线图
plt.figure(figsize=(10, 6))
ax = sns.boxplot(hue = 'RL_number',x = 'LR_main_field', y='cosine_similarity', data=rl_si, palette= [rgb_list[1],rgb_list[3]],linewidth=1,showmeans=True,medianprops= dict(color= rgb_list[0]),meanprops = {'marker':'d','markerfacecolor':rgb_list[4],'markeredgecolor':rgb_list[4]})#'markerfacecolor':'b'
#ax.set_xticklabels(['Basic research','Applied research'])
#ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
ax.set_xticklabels(['SSH','LES','PSE','BHS','MCS'])
handles, labels = plt.gca().get_legend_handles_labels()
labels = ['Basic research','Applied research']
handles = [handles[0],handles[1]]

first_legend = ax.legend(handles, labels, loc='upper left')

#dataset_handles = [plt.Line2D([0], [0] ), 
#                   plt.Line2D([0], [0])]

# Add the existing legend back to the plot
#ax.add_artist(first_legend)
#plt.legend(loc = 'upper right')
plt.legend(handles,labels,loc = 'upper right')

plt.xlabel('Broad research field')
plt.ylabel('Similarity of titles between papers and press releases')
plt.show()

In [ ]:
#heatmap
#transfer into matrix
df = rl_si.groupby(['RL_number','LR_main_field'])['cosine_similarity'].mean().reset_index()
#df = df.sort_values(by='RLEV',ascending=True)
matrix = df.pivot(index='LR_main_field', columns='RL_number', values='cosine_similarity')
# 绘制热力图
plt.figure(figsize=(10, 6))
x_labels = ['Basic research','Applied research']
#sns.heatmap(matrix, annot=True, cmap='Blues')#'viridis'
sns.heatmap(matrix, annot=True, cmap='Blues',xticklabels=x_labels)# 
plt.xticks(rotation=0)
# 添加标题和标签
#plt.title('Heatmap of 3-Column Data')

plt.xlabel('Research Level')
plt.ylabel('Broad research field')
plt.savefig('D:/Users/zhangz10/research level/data/heatmap_high_res.png', dpi=300, bbox_inches='tight', format='png')
# 显示热力图
plt.show()


In [ ]:
---------------Read ease-------------

In [51]:
import textstat
import pronouncing
import numpy as np
import nltk
from nltk import word_tokenize, pos_tag
import spacy
from nltk.corpus import words
common_words = set(words.words())
# 加载英语模型
nlp = spacy.load("en_core_web_sm")
textstat.set_lang("en")
nltk.download('words')

[nltk_data] Downloading package words to \\VUW\Personal$\Homes\Z\zhang
[nltk_data]     z10\Appdata\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [52]:
#计算文本的可读性
def segment_sentences(text):
    sentences = nltk.sent_tokenize(text)
    return sentences 


def calculate_readability(text):
    try:
        # Flesch Reading Ease
        flesch_reading_ease = textstat.flesch_reading_ease(text)     #[0-100]
        #grade_level = textstat.flesch_kincaid_grade(text)
        gunning_fog_index = textstat.gunning_fog(text)
        automated_readability_index = textstat.textstat.smog_index(text)
        coleman_liau_index = textstat.coleman_liau_index(text)
        
        return flesch_reading_ease,gunning_fog_index,automated_readability_index,coleman_liau_index
    
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [53]:
sql1 = "select a.euid,a.RL,a.RL_number,b.content,b.abstract from costas_rlev_out a inner join eu_openalex_LR b on a.euid = b.euid"
data_eu = sql_df(sql1)

In [56]:
full_complex = [['euid','eu_flesch','eu_gunning','eu_automated','eu_coleman', 'paper_flesch','paper_gunning','paper_automated','paper_coleman']]
file1 = open('D:/Users/zhangz10/research level/data/eurekalert_sen_readability.csv', 'a+', newline='', encoding='utf-8')
writer1= csv.writer(file1, delimiter=';')
writer1.writerows(full_complex)
file1.close()

In [ ]:
#readability
for i in tqdm(range(0,len(data_eu))): #len(df_plos)
    
    euid = data_eu['euid'][i]
    content = data_eu['content'][i].replace('\u202f','').replace('\xa0 ','').replace('\xa0','').replace('###','')
    abstract = data_eu['abstract'][i]
    eu_flesch,eu_gunning,eu_automated,eu_coleman = calculate_readability(content)
    paper_flesch,paper_gunning,paper_automated,paper_coleman = calculate_readability(abstract)
    

    paper_fulltext = [[euid,eu_flesch,eu_gunning,eu_automated,eu_coleman, paper_flesch,paper_gunning,paper_automated,paper_coleman]]
    file2 = open('D:/Users/zhangz10/research level/data/eurekalert_sen_readability.csv', 'a+', newline='', encoding='utf-8')
    writer2= csv.writer(file2, delimiter=';')
    writer2.writerows(paper_fulltext)
    file2.close() 
    

In [ ]:
--------------------------analysis data_eu------------

In [117]:
#abstract
f2 = open('D:/Users/zhangz10/research level/data/eurekalert_sen_readability.csv',encoding='utf-8-sig')
df1 = pd.read_csv(f2,sep=';')
f2.close()

In [118]:
#readability
rl_read = pd.merge(df1,data_rl, on='euid', how='inner' )


In [ ]:
#Figure 7
fig = plt.figure(figsize=(12,6))  # create a figure

#labels = ['Basic research','Applied research']
labels = ['Basic research','Applied research']
#
ax1 = fig.add_subplot(1, 2, 1)

#transfer into matrix
df = rl_read.groupby(['RL','LR_main_field'])['paper_flesch'].mean().reset_index()
#df = df.sort_values(by='RLEV',ascending=True)
matrix = df.pivot(index='LR_main_field', columns='RL', values='paper_flesch')
matrix = matrix[matrix.columns[::-1]]

# plot heatmap of paper abstrasts
x_labels = ['Basic research','Applied research']
y_lables = ['BHS','LES','MCS','PSE','SSH']
#sns.heatmap(matrix, annot=True, cmap='Blues')#'viridis'
ax1 = sns.heatmap(matrix, annot=True, cmap='Blues', xticklabels=x_labels,yticklabels=y_lables,vmin=30, vmax=50)

#ax1.yticks(rotation=0)
ax1.set_xlabel('Research Level')
ax1.set_ylabel('Broad research field')
ax1.set_title('Readability of paper abstracts')


ax2 = fig.add_subplot(1, 2, 2)

#transfer into matrix
df = rl_read.groupby(['RL','LR_main_field'])['eu_flesch'].mean().reset_index()
#df = df.sort_values(by='RLEV',ascending=True)
matrix = df.pivot(index='LR_main_field', columns='RL', values='eu_flesch')
matrix = matrix[matrix.columns[::-1]]

# plot heatmap of press releases
x_labels = ['Basic research','Applied research']
y_lables = ['BHS','LES','MCS','PSE','SSH']
#sns.heatmap(matrix, annot=True, cmap='Blues')#'viridis'
ax2 = sns.heatmap(matrix, annot=True, cmap='Blues', xticklabels=x_labels,yticklabels=y_lables,vmin=30, vmax=50)

ax2.set_xlabel('Research Level')
ax2.set_ylabel('Broad research field')
ax2.set_title('Readability of full text of press releases')



plt.tight_layout()
plt.show()

In [ ]:
---------------TOP journal,author,institution---------

In [102]:
f2 = open('D:/Users/zhangz10/research level/data/top_journal.csv')
df_top = pd.read_csv(f2,sep=';')
f2.close()
df = df_top.groupby(['journal','RL'])['number'].sum().reset_index()
top_data = df.pivot(index='journal', columns='RL', values='number')

In [104]:
f2 = open('D:/Users/zhangz10/research level/data/top_institution.csv')
df_top = pd.read_csv(f2,sep=';')
f2.close()
df = df_top.groupby(['institution','RL'])['number'].sum().reset_index()
top_data = df.pivot(index='institution', columns='RL', values='number')

In [52]:
f2 = open('D:/Users/zhangz10/research level/data/top_organization.csv')
df_top = pd.read_csv(f2,sep=';')
f2.close()
df = df_top.groupby(['author_institution','RL'])['number'].sum().reset_index()
top_data = df.pivot(index='author_institution', columns='RL', values='number')

In [ ]:
#Figure 8,9,10
#set plt and colors
sns.set(style="whitegrid")


# 使用 Seaborn 画箱线图
plt.figure(figsize=(12, 6))

# 计算总和并排序
top_data['Total'] = top_data[['Applied research', 'Basic research']].sum(axis=1)
top_data = top_data.sort_values('Total', ascending=True)

# 删除 'Total' 列
top_data = top_data.drop(columns='Total')
top_data = top_data[['Basic research', 'Applied research']]

# 绘制横向堆积柱状图
ax = top_data.plot(kind='barh', stacked=True,color=[rgb_list[1],rgb_list[3]])

# 添加标题和标签
plt.xlabel('Number of press releases (2015-2022)')
plt.ylabel('Institutions issuing press releases')
#plt.ylabel('Journals')
#plt.ylabel('Author affiliations of publications in press releases')
# 添加图例
plt.legend()
#plt.legend(title='Subcategories', bbox_to_anchor=(1.05, 1), loc='upper left')

# 显示图表
plt.show()